### Multi-layer perceptron on mnist

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  
print ("PACKAGES LOADED")

PACKAGES LOADED


### Download and extract mnist dataset

In [3]:
### Download and extract mnist dataset

mnist = input_data.read_data_sets('../Data/', one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print('MNIST loaded')

Extracting ../Data/train-images-idx3-ubyte.gz
Extracting ../Data/train-labels-idx1-ubyte.gz
Extracting ../Data/t10k-images-idx3-ubyte.gz
Extracting ../Data/t10k-labels-idx1-ubyte.gz
MNIST loaded


### Model definition

In [4]:
# NETWORK TOPOLOGIES
n_hidden_1 = 256
n_hidden_2 = 128
n_input    = 784
n_classes  = 10

# INPUTS AND OUTPUTS
x = tf.placeholder(tf.float32, shape=[None, n_input])
y = tf.placeholder(tf.float32, shape=[None, n_classes])

#NETWORK PARAMETERS
stddev = 0.1
weights = {
    'h1' : tf.Variable(tf.random_normal(shape=[n_input,    n_hidden_1], stddev=stddev)),
    'h2' : tf.Variable(tf.random_normal(shape=[n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal(shape=[n_hidden_2, n_classes],  stddev=stddev))
}

bias = {
    'b1' : tf.Variable(tf.random_normal(shape=[n_hidden_1])),
    'b2' : tf.Variable(tf.random_normal(shape=[n_hidden_2])),
    'out': tf.Variable(tf.random_normal(shape=[n_classes]))
}

print('Model definition completed') # Check the grammar error above

Model definition completed


### MLP as a function

In [5]:
def MLP(X_, weights_, bias_):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(X_, weights_['h1']), bias_['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights_['h2']), bias_['b2']))
    out     = tf.add(tf.matmul(layer_2, weights_['out']), bias_['out'])
    return out

### Create Tensor graph for ANN

In [6]:
# Prediction
pred = MLP(x, weights, bias)

# Loss and Optimizer
# Do calculate C/E with logits.. not with softmax values
cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y)
cost = tf.reduce_mean(cost)

optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, tf.float32))

# Initializer
init = tf.global_variables_initializer()

print('Tensor graph completed')

Tensor graph completed


### Train the model

In [9]:
training_epochs = 50
batch_size      = 100
display_step    = 5

# Session
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(init)

In [10]:
# Minibatch learning
for epoch in range(training_epochs): # Epochs
    avg_cost = 0.0
    num_batch = int(mnist.train.num_examples / batch_size)
    for i in range(num_batch): # Iteration
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        feeds = {x:batch_xs, y:batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds) / num_batch
    
    # Display training steps
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test  = {x: mnist.test.images, y:mnist.test.labels}
        
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc  = sess.run(accr, feed_dict=feeds_test)
        
        print('Epoch: %03d/%03d cost: %0.9f train_acc: %.3f test_acc %.3f'
             % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print('Done')

Epoch: 000/050 cost: 0.284061253 train_acc: 0.960 test_acc 0.960
Epoch: 005/050 cost: 0.018922309 train_acc: 0.990 test_acc 0.974
Epoch: 010/050 cost: 0.005605694 train_acc: 1.000 test_acc 0.980
Epoch: 015/050 cost: 0.001768287 train_acc: 1.000 test_acc 0.975
Epoch: 020/050 cost: 0.002039959 train_acc: 1.000 test_acc 0.977
Epoch: 025/050 cost: 0.001783123 train_acc: 1.000 test_acc 0.980
Epoch: 030/050 cost: 0.001218296 train_acc: 1.000 test_acc 0.980
Epoch: 035/050 cost: 0.000575635 train_acc: 1.000 test_acc 0.980
Epoch: 040/050 cost: 0.000620703 train_acc: 1.000 test_acc 0.980
Epoch: 045/050 cost: 0.000447184 train_acc: 1.000 test_acc 0.980
Done
